## import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

## 실행환경 설정

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

## epochs, batch의 크기 설정

In [3]:
EPOCHS = 40
BATCH_SIZE = 64
EPOCHS = 40
BATCH_SIZE = 64

## 학습용, 테스트용 데이터셋 설정

In [9]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                         train=True,
                         download=True,
                         transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                         ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                         train=False,
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,), (0.3081,))
                         ])),
    batch_size=BATCH_SIZE, shuffle=True)

## CNN 모듈 생성

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2(x),2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## CNN 모델의 인스턴스와 최적화 함수

In [16]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

## train 함수 

In [25]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]/tLoss:{:.6f}'
                 .format(epoch, batch_idx * len(data),
                        len(train_loader.dataset),
                        100. * batch_idx / len(train_loader),
                        loss.item()))

## evaluate 함수

In [26]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target,
                                        reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 학습 시작

In [27]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss : {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]/tLoss:2.308420
Train Epoch: 1 [12800/60000 (21%)]/tLoss:0.736929
Train Epoch: 1 [25600/60000 (43%)]/tLoss:0.519441
Train Epoch: 1 [38400/60000 (64%)]/tLoss:0.264947
Train Epoch: 1 [51200/60000 (85%)]/tLoss:0.489661
[1] Test Loss : 0.1493, Accuracy: 95.32%
Train Epoch: 2 [0/60000 (0%)]/tLoss:0.310312
Train Epoch: 2 [12800/60000 (21%)]/tLoss:0.293277
Train Epoch: 2 [25600/60000 (43%)]/tLoss:0.171607
Train Epoch: 2 [38400/60000 (64%)]/tLoss:0.337428
Train Epoch: 2 [51200/60000 (85%)]/tLoss:0.142978
[2] Test Loss : 0.0864, Accuracy: 97.24%
Train Epoch: 3 [0/60000 (0%)]/tLoss:0.176053
Train Epoch: 3 [12800/60000 (21%)]/tLoss:0.131970
Train Epoch: 3 [25600/60000 (43%)]/tLoss:0.174874
Train Epoch: 3 [38400/60000 (64%)]/tLoss:0.284700
Train Epoch: 3 [51200/60000 (85%)]/tLoss:0.139278
[3] Test Loss : 0.0672, Accuracy: 97.76%
Train Epoch: 4 [0/60000 (0%)]/tLoss:0.151322
Train Epoch: 4 [12800/60000 (21%)]/tLoss:0.339980
Train Epoch: 4 [25600/60000 (43%)]/tLoss:0.1900

Train Epoch: 29 [25600/60000 (43%)]/tLoss:0.102782
Train Epoch: 29 [38400/60000 (64%)]/tLoss:0.099925
Train Epoch: 29 [51200/60000 (85%)]/tLoss:0.086040
[29] Test Loss : 0.0291, Accuracy: 98.99%
Train Epoch: 30 [0/60000 (0%)]/tLoss:0.006011
Train Epoch: 30 [12800/60000 (21%)]/tLoss:0.053490
Train Epoch: 30 [25600/60000 (43%)]/tLoss:0.026828
Train Epoch: 30 [38400/60000 (64%)]/tLoss:0.011598
Train Epoch: 30 [51200/60000 (85%)]/tLoss:0.058428
[30] Test Loss : 0.0270, Accuracy: 99.09%
Train Epoch: 31 [0/60000 (0%)]/tLoss:0.017703
Train Epoch: 31 [12800/60000 (21%)]/tLoss:0.178803
Train Epoch: 31 [25600/60000 (43%)]/tLoss:0.001138
Train Epoch: 31 [38400/60000 (64%)]/tLoss:0.014760
Train Epoch: 31 [51200/60000 (85%)]/tLoss:0.018188
[31] Test Loss : 0.0261, Accuracy: 99.17%
Train Epoch: 32 [0/60000 (0%)]/tLoss:0.031514
Train Epoch: 32 [12800/60000 (21%)]/tLoss:0.020844
Train Epoch: 32 [25600/60000 (43%)]/tLoss:0.054478
Train Epoch: 32 [38400/60000 (64%)]/tLoss:0.137132
Train Epoch: 32 [51200